## Advantage actor-critic in AgentNet (5 pts)

Once we're done with REINFORCE, it's time to proceed with something more sophisticated.
The next one in line is advantage actor-critic, in which agent learns both policy and value function, using the latter to speed up learning.


In [1]:
%env THEANO_FLAGS='floatX=float32'
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1
        
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


env: THEANO_FLAGS='floatX=float32'


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import gym

env = gym.make("MountainCar-v0")
obs = env.reset()
state_size = len(obs)
n_actions = env.action_space.n
print(obs)

[2017-03-15 03:02:34,564] Making new env: MountainCar-v0


[-0.40807951  0.        ]


# Basic agent setup
Here we define a simple agent that maps game images into Qvalues using shallow neural network.


In [3]:
import lasagne
from lasagne.layers import InputLayer,DenseLayer,NonlinearityLayer,batch_norm,dropout
#image observation at current tick goes here, shape = (sample_i,x,y,color)
observation_layer = InputLayer((None,state_size))

nn = DenseLayer(observation_layer,256)#<your architecture>
nn = DenseLayer(nn,256)#<your architecture>

Couldn't import dot_parser, loading of dot files will not be possible.


In [4]:
#a layer that predicts Qvalues

policy_layer = DenseLayer(nn,n_actions,nonlinearity=lasagne.nonlinearities.softmax)#<estimate probabilities of actions given prev layer. Mind the nonlinearity!>


V_layer = DenseLayer(nn,1,nonlinearity=None)#<estimate state values (1 unit layer). Mind nonlinearity too.>

In [5]:
#To pick actions, we use an epsilon-greedy resolver (epsilon is a property)
from agentnet.resolver import ProbabilisticResolver
action_layer = ProbabilisticResolver(policy_layer,
                                     name="e-greedy action picker",
                                     assume_normalized=True)

##### Finally, agent
We declare that this network is and MDP agent with such and such inputs, states and outputs

In [6]:
from agentnet.agent import Agent
#all together
agent = Agent(observation_layers=observation_layer,
              policy_estimators=(policy_layer,V_layer),
              action_layers=action_layer)


In [7]:
#Since it's a single lasagne network, one can get it's weights, output, etc
weights = lasagne.layers.get_all_params((action_layer,V_layer),trainable=True)
weights

[W, b, W, b, W, b, W, b]

# Create and manage a pool of atari sessions to play with

* To make training more stable, we shall have an entire batch of game sessions each happening independent of others
* Why several parallel agents help training: http://arxiv.org/pdf/1602.01783v1.pdf
* Alternative approach: store more sessions: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [8]:
from agentnet.experiments.openai_gym.pool import EnvPool

#create a small pool with 10 parallel agents
pool = EnvPool(agent,"MountainCar-v0", n_games=10,max_size=1000) 

#we assume that pool size 1000 is small enough to learn "almost on policy" :)

[2017-03-15 03:02:35,582] Making new env: MountainCar-v0
[2017-03-15 03:02:35,585] Making new env: MountainCar-v0
[2017-03-15 03:02:35,587] Making new env: MountainCar-v0
[2017-03-15 03:02:35,589] Making new env: MountainCar-v0
[2017-03-15 03:02:35,591] Making new env: MountainCar-v0
[2017-03-15 03:02:35,593] Making new env: MountainCar-v0
[2017-03-15 03:02:35,595] Making new env: MountainCar-v0
[2017-03-15 03:02:35,598] Making new env: MountainCar-v0
[2017-03-15 03:02:35,600] Making new env: MountainCar-v0
[2017-03-15 03:02:35,602] Making new env: MountainCar-v0


In [9]:
%%time
#interact for 7 ticks
_,action_log,reward_log,_,_,_  = pool.interact(7)


print(action_log[:3])
print(reward_log[:3])

[[2 0 1 2 1 0 0]
 [0 2 0 2 0 2 1]
 [0 1 2 1 0 1 0]]
[[-1. -1. -1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1. -1.  0.]
 [-1. -1. -1. -1. -1. -1.  0.]]
CPU times: user 7.16 ms, sys: 0 ns, total: 7.16 ms
Wall time: 6.97 ms


In [10]:
SEQ_LENGTH = 10
#load first sessions (this function calls interact and remembers sessions)
pool.update(SEQ_LENGTH)

# Actor-critic loss

Here we define obective function for actor-critic (one-step) RL.

* We regularize policy with expected inverse action probabilities (discouraging very small probas) to make objective numerically stable


In [11]:
#get agent's Qvalues obtained via experience replay
replay = pool.experience_replay

_,_,_,_,(policy_seq,V_seq) = agent.get_sessions(
    replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,
)



In [12]:
from agentnet.learning import a2c                                                   


elwise_mse_loss = a2c.get_elementwise_objective(policy_seq,
                                                V_seq[:,:,0],
                                                replay.actions[0],
                                                replay.rewards,
                                                replay.is_alive,
                                                gamma_or_gammas=0.99,
                                                n_steps=1)

#compute mean over "alive" fragments
loss = elwise_mse_loss.sum() / replay.is_alive.sum()

In [13]:
from theano import tensor as T
reg_entropy = (1./(policy_seq)).sum(-1).mean()#<regularize agent with 1/pi. Higher entropy = smaller loss. >

loss += 0.01*reg_entropy

In [14]:
# Compute weight updates
updates = lasagne.updates.rmsprop(loss,weights,learning_rate=0.001)

In [15]:
import theano
train_step = theano.function([],loss,updates=updates)

# Demo run

In [16]:
#for MountainCar-v0 evaluation session is cropped to 200 ticks
untrained_reward = pool.evaluate(save_path="./records",record_video=True)

#video is in the ./records folder

[2017-03-15 03:02:44,437] Making new env: MountainCar-v0
[2017-03-15 03:02:44,440] DEPRECATION WARNING: env.spec.timestep_limit has been deprecated. Replace your call to `env.spec.timestep_limit` with `env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')`. This change was made 12/28/2016 and is included in version 0.7.0
[2017-03-15 03:02:44,441] Clearing 3 monitor files from previous run (because force=True was provided)
[2017-03-15 03:02:44,443] Starting new video recorder writing to /root/anet/Practical_RL/week6/records/openaigym.video.0.3558.video000000.mp4
[2017-03-15 03:03:11,264] Tried to pass invalid video frame, marking as broken: Your frame has shape (1, 1, 3), but the VideoRecorder is configured for shape (400, 600, 3).
[2017-03-15 03:03:11,362] Cleaning up paths for broken video recorder: path=/root/anet/Practical_RL/week6/records/openaigym.video.0.3558.video000000.mp4 metadata_path=/root/anet/Practical_RL/week6/records/openaigym.video.0.3558.video000000.meta.json

Episode finished after 200 timesteps with reward=-200.0


# Training loop


In [ ]:
#starting epoch
epoch_counter = 1

#full game rewards
rewards = {}

In [ ]:
from tqdm import tqdm
#the loop may take eons to finish.
#consider interrupting early.
loss = 0
for i in tqdm(range(10000)):    
    
    #train
    pool.update(SEQ_LENGTH,append=True)
    
    loss = loss*0.99 + train_step()*0.01
        
    
    
    if epoch_counter%100==0:
        #average reward per game tick in current experience replay pool
        pool_mean_reward = np.average(pool.experience_replay.rewards.get_value()[:,:-1],
                                      weights=1+pool.experience_replay.is_alive.get_value()[:,:-1])
        print("iter=%i\treward/step=%.5f\tloss ma=%.5f"%(epoch_counter,
                                                        pool_mean_reward,
                                                        loss))
        

    ##record current learning progress and show learning curves
    if epoch_counter%500 ==0:
        n_games = 10
        rewards[epoch_counter] = pool.evaluate( record_video=False,n_games=n_games,
                                               verbose=False)
        print("Current score(mean over %i) = %.3f"%(n_games,np.mean(rewards[epoch_counter])))
    
    
    epoch_counter  +=1

    
# Time to drink some coffee!

  1%|          | 101/10000 [00:06<18:53,  8.73it/s]

iter=100	reward/step=-1.00000	loss ma=-0.12804


  2%|▏         | 201/10000 [00:19<19:20,  8.45it/s]

iter=200	reward/step=-1.00000	loss ma=0.50704


  3%|▎         | 301/10000 [00:31<19:14,  8.40it/s]

...

 53%|█████▎    | 5301/10000 [11:05<09:11,  8.52it/s]

iter=5300	reward/step=-0.99721	loss ma=0.45238


 54%|█████▍    | 5401/10000 [11:17<09:01,  8.50it/s]

iter=5400	reward/step=-0.99716	loss ma=0.45623


 54%|█████▍    | 5415/10000 [11:19<08:55,  8.57it/s]

In [ ]:
iters,session_rewards=zip(*sorted(rewards.items(),key=lambda (k,v):k))

In [ ]:
plt.plot(iters,map(np.mean,session_rewards))

### Visualizing the $V(s)$ and  $\pi(a|s)$

Since the observation space is just 2-dimensional, we can plot it on a 2d scatter-plot to gain insight of what agent learned.

In [ ]:
_,_,_,_,(pool_policy,pool_V) = agent.get_sessions(
    pool.experience_replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,)

plt.scatter(
    *pool.experience_replay.observations[0].get_value().reshape([-1,2]).T,
    c = pool_V.ravel().eval(),
    alpha = 0.1)
plt.title("predicted state values")
plt.xlabel("position")
plt.ylabel("speed")

In [ ]:
obs_x,obs_y = pool.experience_replay.observations[0].get_value().reshape([-1,2]).T
optimal_actid = pool_policy.argmax(-1).ravel().eval()
action_names=["left","stop","right"]
for i in range(3):
    sel = optimal_actid==i
    plt.scatter(obs_x[sel],obs_y[sel],
                c=['red','blue','green'][i],
                alpha = 0.1,label=action_names[i])
    
plt.title("most likely action id")
plt.xlabel("position")
plt.ylabel("speed")
plt.legend(loc='best')

### Variations in the algorithm (2 pts)

Try different `n_steps` param to see if it improves learning performance.

Your objective is to compare learning curves for 1, 3, 10 and 25-step updates (or any grid you think is appropriate).

For 25-step updates, please also increase SEQ_LENGTH to 25.

_(Bonus)_ Also evaluate how performance changes with different entropy regularizer coefficient.

In [ ]:
#<a lot of your code here>

### Bonus section (5+ pts)

Beat the [`LunarLanderContinuous-v2`](https://gym.openai.com/envs/LunarLanderContinuous-v2) with continuous version of advantage actor-critic.

You will require a multidimensional gaussian (or similar) policy from your agent.

You can implement that by feeding a2c.get_elementwise_objective probabilities of agent's chosen actions (it will be 2-dimensional) instead of all actions.

Contact us if you have any questions.